# El commentator

In [2]:
import functools
import os
import pandas as pd
import tabula
import re
from unidecode import unidecode

## Dataframe extraction for a single pdf file

In [39]:
pdf_path = "./Bulletins/AAZZOUZ Bilel-Trimestre 2-1G2-N-6569.pdf"
pdf_path2 = "./Bulletins/AAZZOUZ Bilel-Trimestre 3-2NDE 7-N-3597.pdf"

df = tabula.read_pdf(
    pdf_path,
     pages=1,
     stream=False,
     lattice=True,
     guess=True,
     )[0]

# Drop 'field-name' column & last NaN column 
df = df.drop([df.columns[0], df.columns[6]], axis=1) 

# Drop rows with no comment
df = df.dropna(subset=[df.columns[4]]) 


df.columns = ['eleve', 'classe', "min", "max", "commentaire"]

convert_dict = {
    'eleve': float,
    'classe': float,
    'min': float,
    'max': float,
    'commentaire': str
 }

# Pre-process columns 
df = df.apply(lambda x: x.replace(',','.', regex=True) if x.name != 'commentaire' else x, axis=1)
## Filter rows with multiple student grades values
df = df[~df['eleve'].str.contains('\r')]

df = df.astype(convert_dict)

# Remove returns in comments
df['commentaire'] = df['commentaire'].str.replace('\r',' ', regex=True)

df

"Les résultats sont devenus insuffisants. faute de travail. L'attention en classe est problématique et la participation inexistante. Il faut vous ressaisir !"

## Extract raw data for whole directory

In [73]:
directory = './Bulletins'
files = os.listdir(directory)

col_names = ['eleve', 'classe', "min", "max", "commentaire"]

def get_datastream_from_file(path):
  df = tabula.read_pdf(
      path,
      pages=1,
      stream=False,
      lattice=True,
      guess=True,
      )[0]

  df = df.drop([df.columns[0], df.columns[6]], axis=1)
  df = df.dropna()

  df.columns = col_names

  return df


files_subset = files
files_path = list(map(lambda name : directory + '/' + name, files_subset))

result = pd.DataFrame(columns=col_names)
for file_path in files_path:
  result = result.append(get_datastream_from_file(file_path))

result.to_csv('data_raw.csv', index=None)

## Extract data from google docs

In [133]:
df = pd.read_clipboard()
df.head()

,eleve,classe,min,max,commentaire
0,5.0,"9,06",0,19.5,Ensemble demeuré fragile.
1,8.0,"9,06",0,19.5,Ensemble fragile.
2,11.5,"9,06",0,19.5,Ensemble honorable.
3,3.5,"9,06",0,19.5,Ensemble demeuré fragile.
4,12.5,"9,06",0,19.5,Ensemble en recul mais qui demeure honorable.


In [134]:
df.shape

(96, 5)

In [135]:
df.to_csv('data_complete.csv', mode='a', header=None, index=None)

## Cleanup 

In [223]:
df = pd.read_csv('data_complete.csv')
df.head()

,Unnamed: 0,eleve,classe,min,max,commentaire
0,0,"18,29","12,12","5,13","18,29",Un anglais de qualité (tant à l'oral qu'à l'éc...
1,1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide,..."
2,2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,4,"10,60","10,87","5,47","16,71",Résultats corrects mais le comportement en cla...


### Indexing
data_complete has messy indexing

In [224]:
df = df.drop([df.columns[0]], axis=1)
df.head()

,eleve,classe,min,max,commentaire
0,"18,29","12,12","5,13","18,29",Un anglais de qualité (tant à l'oral qu'à l'éc...
1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide,..."
2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,"10,60","10,87","5,47","16,71",Résultats corrects mais le comportement en cla...


In [225]:
df.to_csv('data_clean.csv', index=None)

## Read

In [226]:
df = pd.read_csv('data_clean.csv')

In [231]:
df['commentaire'].head()

0    Un anglais de qualité (tant à l'oral qu'à l'éc...
1    Niveau d'anglais excellent. C'est très solide,...
2       Un excellent début d'année, poursuivez ainsi !
3                                        Bon ensemble.
4    Résultats corrects mais le comportement en cla...
Name: commentaire, dtype: object

In [232]:
df.shape

(9357, 5)

### Nan values

In [233]:
# Drop na values from 'commentaire' column
df = pd.read_csv('data_clean.csv')
df2 = df.dropna(axis="index", how="any", subset=['commentaire'])

In [234]:
# Other columns don't have any na value left
cols = ['eleve', 'classe','min', 'max']
[sum(df2[col].isna()) for col in cols]

[0, 0, 0, 0]

In [ ]:
df2.isna().any()

In [235]:
df.shape

(9357, 5)

In [236]:
df2.shape

(8099, 5)

In [237]:
df2.to_csv('data_clean.csv', index=None)

### Grades cleanup

In [238]:
df = pd.read_csv('data_clean.csv')
df.head()

,eleve,classe,min,max,commentaire
0,"18,29","12,12","5,13","18,29",Un anglais de qualité (tant à l'oral qu'à l'éc...
1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide,..."
2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,"10,60","10,87","5,47","16,71",Résultats corrects mais le comportement en cla...


#### Multiple grades

In [239]:
# How many rows with multiple grades
mask = df['eleve'].str.contains('\r')
mask.sum()

176

In [240]:
# Filter these
df = df[~mask]
df.to_csv('data_clean.csv', index=False)

In [241]:
# Remaining non numerical values are semantic
df[~df['eleve'].str.contains('\d+[\.,]*\d*', case=False, regex=True)].value_counts(subset=['eleve'])

eleve
Disp     42
Abs      36
N.Not    33
Inap      7
dtype: int64

#### Formating numbers

In [242]:
cols = ['eleve', 'classe','min', 'max']
for col in cols:
    df[col] = df[col].str.replace(',','.', regex=True)

In [ ]:
df = df.astype({"classe": 'float16', "min": 'float16', "max": 'float16'})
df.info()

In [243]:
df.shape

(7923, 5)

### Commentaires cleanup

In [260]:
# remove new line \r returns
mask = df['commentaire'].str.contains('\r')
len(df[mask])

0

In [256]:
df['commentaire'] = df['commentaire'].str.replace('\r',' ', regex=True)

## Filter comments with explicit field name

In [262]:
# Trouver le nombre d'appréciations contenant un mot clé 
keywords = ["math", "phys", 'anglais', "françai", "sport", "natation", "judo", "basket", "course", "badmin", "yoga", "escalade", "littéra", "llce", "philo", "espagn", "allema", "italie", "musical", "musiq", "svt", "blouse", "scienc", "tp", "eps", "histoir", "géog", "dnl", "théât", "latin", "grec", "MPS"]
found = {}
for keyword in keywords: 
    mask = df['commentaire'].str.contains(keyword, case=False, regex=False)
    found[keyword] = len(df.loc[mask, "commentaire"])

found

{'math': 52,
 'phys': 12,
 'anglais': 84,
 'françai': 34,
 'sport': 6,
 'natation': 7,
 'judo': 14,
 'basket': 7,
 'course': 4,
 'badmin': 13,
 'yoga': 4,
 'escalade': 25,
 'littéra': 4,
 'philo': 0,
 'espagn': 13,
 'allema': 2,
 'italie': 0,
 'musical': 12,
 'musiq': 0,
 'svt': 48,
 'scienc': 18,
 'tp': 11,
 'eps': 10,
 'histoir': 6,
 'géog': 5,
 'dnl': 12,
 'théât': 1,
 'latin': 6,
 'grec': 2,
 'MPS': 47}

In [263]:
# Total d'appréciations contenant le nom de la discipline
functools.reduce(lambda x,y: x+y, found.values(), 0)

459

In [264]:
# Filter ces appréciations
pattern = "|".join(keywords)
mask = df['commentaire'].str.contains(pattern, case=False, regex=True)
df = df.loc[~mask]

df.shape

(7493, 5)

## Filter stutdents names

In [265]:
# Create list of all student names from files
files = os.listdir('./Bulletins')
student_names = set()
for filename in files:
  name = re.search("^(?P<nom>[A-Z\- ]*) (?P<prenom>.*)-Trimestre", filename)
  student_names.add(name.group("nom"))
  student_names.add(name.group("prenom"))

student_names;

In [352]:
# Trouver le nombre d'appréciations contenant un nom d'élève (insensible aux accents)
student_names_found = {}
for name in student_names: 
    mask = df['commentaire'].apply(lambda x: unidecode(x)).str.contains(f" {unidecode(name)} ", case=False, regex=False)
    nb = len(df.loc[mask, "commentaire"])
    if nb > 0:
        student_names_found[name] = nb

In [353]:
student_names_found

{}

In [287]:
# Total des noms d'élèves apparaissant en commentaire
functools.reduce(lambda summ, x: summ+x, student_names_found.values(), 0)

1402

In [349]:
# Fonction qui anonymise une string en cherchant un pattern de noms et les remplaçant par <name>, insensible aux accents et aux majuscules. 
# Le reste de la string est inchangé (reste accentué)
def accent_i_replace(strg, pattern):
    m = re.search(f" {unidecode(pattern)} ", unidecode(strg), re.IGNORECASE)
    
    # Find accented version
    if m:
        accented = strg[m.start():m.end()]
        return strg.replace(accented, ' <name> ')
    else:
        return strg


In [351]:
# Remplace les noms d'élèves (insensible aux accents) dans les appréciations par '<name>'
pattern = "|".join(student_names)
df['commentaire'] = df['commentaire'].apply(lambda x: accent_i_replace(x, pattern))


### Nettoyer les commmentaires tronqués

In [429]:
# Trouve les commentaires incluant des ponctuations mais ne finissant pas par une ponctuation et capture le bout de commentaire depuis la dernière ponctuation
# Pattern first group: capturing (greedy) group ending in punctuation (. ? !)
# Pattern second group : non capturing ignore group (everything else)
# If there is a match, .replace will only keep the 'keep' group. So it keeps one liner comments without punctuation
pattern = "(?P<keep>.*[\.?!])(?:.*)"
df["commentaire"] = df["commentaire"].str.replace(pattern, lambda m: m.group("keep"))

<ipython-input-429-c4b07f24c590>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["commentaire"] = df["commentaire"].str.replace(pattern, lambda m: m.group("keep"))


In [433]:
# Certains i ne sont pas bien passés
df["commentaire"] = df["commentaire"].str.replace("í", "i")

In [20]:
# Il y a quelques caractère russes dans un commentaire (trouvé par inspection des caractères uniques présents dans l'ensemble des commentaires)
list(df.loc[df['commentaire'].str.contains("и"), 'commentaire'])

["C'est un bon trimestre  <name> . Vous êtes plus attentive, vous mûrissez, vous progressez. Continuez ainsi. Pour les friandises exotiques que vous avez partagees avec la classe : сп асибо  !"]

In [21]:
# On filtre le commentaire russe
df = df.loc[~df['commentaire'].str.contains("и")]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7492 entries, 0 to 7492
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   eleve        7492 non-null   object 
 1   classe       7492 non-null   float64
 2   min          7492 non-null   float64
 3   max          7492 non-null   float64
 4   commentaire  7492 non-null   object 
dtypes: float64(3), object(2)
memory usage: 351.2+ KB


## Inspect Disp, Abs, NNot comments

In [432]:
tags = ["Disp", "Abs", "N.Not", "Inap"]
df.loc[df['eleve'] == "Disp", "commentaire"]

66                                Dispensée ce trimestre.
243                                Dispensé ce trimestre.
311                            Dispensée en fin de cycle.
438                               Dispensée ce trimestre.
635                                    Dispensé à l'année
800                            Dispensé pour l'évaluation
988     Dispensée la majeure partie du trimestre,  <na...
1307    Un bon début d'année scolaire. Dispensée en fi...
1348                                             Dispensé
1768                              Dispensée ce trimestre.
1925     <name>  est loin d'exploiter au maximum ses c...
2063    Une petite participation en début de trimestre...
2706                            Dispensé en fin de cycle.
2808    Dispense parentale le jour de l'évaluation... ...
2906                                             Dispensé
2970                           Dispensée en fin de cycle.
3301                              Dispensée ce trimestre.
3564          

## Load

In [3]:
df = pd.read_csv('data_clean.csv')

## Save

In [23]:
df.to_csv('data_clean.csv', index=False)